In [23]:

import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
import os
from tqdm import tqdm
import re
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from datetime import datetime
from nltk.tokenize import word_tokenize, sent_tokenize,RegexpTokenizer
from nltk.corpus import stopwords 


In [24]:
df=pd.read_csv('new_dataset.csv')

In [25]:
df.head()

,Rating,Rev_verify,Review_Date,IC,Reviewer_Name,Review,Rev_summ,Review_timestamp,tokenized_text,key_words,joined_Sent
0,5,True,"09 1, 2015",B009XD5TPQ,Sunny Zoeller,bought it for my husband he's very happy with it,He's very happy with,1441065600,"['bought', 'it', 'for', 'my', 'husband', 'he',...","['bought', 'husband', ""'s"", 'happy']",bought husband 's happy
1,5,True,"01 9, 2016",B016MF3P3K,Denise Lesley,great screen protector doesn't even seem as th...,Five Stars,1452297600,"['great', 'screen', 'protector', 'does', ""n't""...","['great', 'screen', 'protector', ""n't"", 'even'...",great screen protector n't even seem though 's
2,1,True,"01 14, 2016",B00IJJCQBA,Stephanie,to tight on my phone and the bottom piece was ...,One Star,1452729600,"['to', 'tight', 'on', 'my', 'phone', 'and', 't...","['tight', 'phone', 'bottom', 'piece', 'missing']",tight phone bottom piece missing
3,5,False,"06 21, 2017",B00NIJOGOG,SG,very good and superior quality looks great my ...,Very nice and good quality!!!,1498003200,"['very', 'good', 'and', 'superior', 'quality',...","['good', 'superior', 'quality', 'looks', 'grea...",good superior quality looks great wife using l...
4,3,True,"06 10, 2015",B00MQYS97Y,Linda,the charger is not working however the company...,Not happy,1433894400,"['the', 'charger', 'is', 'not', 'working', 'ho...","['charger', 'working', 'however', 'company', '...",charger working however company sending replac...


In [26]:
df1=df[0:30]

In [27]:
df1.drop(['Rev_verify','Rating','Review_Date','Reviewer_Name','Rev_summ','Review_timestamp','tokenized_text','key_words'],axis=1,inplace=True)

In [28]:
df1.drop(['Review'],axis=1,inplace=True)

In [29]:
df1.head()

,IC,joined_Sent
0,B009XD5TPQ,bought husband 's happy
1,B016MF3P3K,great screen protector n't even seem though 's
2,B00IJJCQBA,tight phone bottom piece missing
3,B00NIJOGOG,good superior quality looks great wife using l...
4,B00MQYS97Y,charger working however company sending replac...


In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [31]:

# Document-Term Matrix (Count Vectorizer)
word_vectorizer = CountVectorizer(ngram_range=(1,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(df1["joined_Sent"])
df_doc_term = pd.DataFrame(sparse_matrix.toarray(), columns=word_vectorizer.get_feature_names())
df_doc_term.shape
# df_doc_term.to_csv("df_doc_term.csv", index=True)

(30, 1233)

In [32]:
import time
from sklearn.decomposition import TruncatedSVD

In [33]:
# Topic Modeling with Matrix Factorization and LSA (Latent Semantic Analysis) aka Singular Value Decomposition (SVD)
time1 = time.time()
lsa = TruncatedSVD(3)
# Transform the doc-term matrix to doc-topic matrix
df_doc_topic = lsa.fit_transform(df_doc_term)
lsa.explained_variance_ratio_
# Getting the U-matrix in Decomposition
topic_word = pd.DataFrame(lsa.components_.round(10),   # 10 for decimal places
             index = ["component_1","component_2","component_3"],
             columns = word_vectorizer.get_feature_names()) # notes is vectoriser only 
topic_word
# Top 10 key words for each of the 6 topics
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, word_vectorizer.get_feature_names(), 20)
time2 = time.time()
time_taken = time2 - time1
print('\nTime Taken: {:.2f} seconds'.format(time_taken))


Topic  0
refund, product, well, process, charges, months, last, fast, anker, process refund, support, manager, forgot, anker support, case, good, great, update, since, lightweight

Topic  1
review, charger, way, outlet, product, unit, charging, solid, believe, port, quickly, touch, plugs, horizontally way block, product deserve review, product deserve, say believe giving, ounces oriented, ounces oriented plug, rating product

Topic  2
case, pictures, flash, take, take pictures, good, looking, protection, try, though, turned, iphone, still, great, really, love, lot, ahead, ahead try take, camera issue

Time Taken: 10.50 seconds


In [46]:
  
# dropping ALL duplicte values 
df.drop_duplicates(subset ="IC", 
                     keep = False, inplace = True) 
  


# Deep Learning
To classify ratings<4 as sentiment, i.e. replace ratings less than 4 as not happy

label 1= Happy

label 2= Unhappy

In [36]:
df['sentiment'] = df['Rating']<4

In [37]:
df

,Rating,Rev_verify,Review_Date,IC,Reviewer_Name,Review,Rev_summ,Review_timestamp,tokenized_text,key_words,joined_Sent,sentiment
1043,2,False,"09 7, 2017",B00OXRG9EW,Daniel Rose,arrived promptly looks great but the power but...,"Looks great, but the power button cover isn't ...",1504742400,"['arrived', 'promptly', 'looks', 'great', 'but...","['arrived', 'promptly', 'looks', 'great', 'pow...",arrived promptly looks great power button cove...,True
1982,4,True,"12 30, 2012",B001DQPCR6,mrxtc007_2k2,cable worked just fine for my mom's old phone ...,Works fine!,1356825600,"['cable', 'worked', 'just', 'fine', 'for', 'my...","['cable', 'worked', 'fine', 'mom', ""'s"", 'old'...",cable worked fine mom 's old phone used transf...,False
3450,5,True,"01 13, 2016",B000VSHMF2,Frenchy,very nice,VERY NICE!!!,1452643200,"['very', 'nice']",['nice'],nice,False
4950,5,True,"05 12, 2007",B000NK7ELU,Eagles Point,after all the different bt headsets i have tri...,Excellent choice,1178928000,"['after', 'all', 'the', 'different', 'bt', 'he...","['different', 'bt', 'headsets', 'tried', 'foun...",different bt headsets tried found motorola son...,False
5218,5,True,"09 6, 2016",B00R1L3BL6,Rodrigo Torres,great case love it,Five Stars,1473120000,"['great', 'case', 'love', 'it']","['great', 'case', 'love']",great case love,False
5712,5,True,"09 25, 2015",B00X494RJY,ctx9229,this radio is very amazing it is easy to conne...,"This radio is VERY amazing, it is easy to connect",1443139200,"['this', 'radio', 'is', 'very', 'amazing', 'it...","['radio', 'amazing', 'easy', 'connect', 'instr...",radio amazing easy connect instructions pretty...,False
5881,4,True,"06 8, 2014",B00D41QUJC,Rodwil,i like this wallet case because give the chanc...,Like it,1402185600,"['i', 'like', 'this', 'wallet', 'case', 'becau...","['like', 'wallet', 'case', 'give', 'chance', '...",like wallet case give chance carry around phon...,False
6451,1,False,"04 11, 2010",B0016569I0,Hieronymus,i am not the only person i have known to have ...,Junk phone; does not last,1270944000,"['i', 'am', 'not', 'the', 'only', 'person', 'i...","['person', 'known', 'model', 'phone', 'fail', ...",person known model phone fail quickly replaced...,True
8675,5,True,"04 21, 2017",B01DHH3K7W,LISA LOU,looks nice especially from a distance i droppe...,Cute,1492732800,"['looks', 'nice', 'especially', 'from', 'a', '...","['looks', 'nice', 'especially', 'distance', 'd...",looks nice especially distance dropped phone t...,False
11563,2,False,"09 27, 2014",B00A474Z4Q,Christopher Champion,it was cool until it broke off while in my poc...,didn't work for me,1411776000,"['it', 'was', 'cool', 'until', 'it', 'broke', ...","['cool', 'broke', 'pocket', 'leaving', 'plug',...",cool broke pocket leaving plug stick inside he...,True


In [38]:

from sklearn.model_selection import train_test_split
train_text, test_text, train_y, test_y = train_test_split(df['Review'],df['sentiment'],test_size = 0.2)

In [39]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

In [40]:

MAX_NB_WORDS = 20000

# get the raw text data
texts_train = train_text.astype(str)
texts_test = test_text.astype(str)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2572 unique tokens.


In [41]:

MAX_SEQUENCE_LENGTH = 200
#pad sequences are used to bring all sentences to same size.
# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

Shape of data tensor: (345, 200)
Shape of data test tensor: (87, 200)


In [42]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,input_shape=(1,)))
model.add(Dense(1, activation='sigmoid'))

In [43]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
model.fit(x_train, train_y,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, test_y))

Train on 345 samples, validate on 87 samples
Epoch 1/5
345/345 [==============================] - ETA: 50s - loss: 0.6912 - accuracy: 0.656 - ETA: 10s - loss: 0.6873 - accuracy: 0.648 - 34s 99ms/step - loss: 0.6814 - accuracy: 0.6870 - val_loss: 0.6462 - val_accuracy: 0.8276
Epoch 2/5
345/345 [==============================] - ETA: 2s - loss: 0.6609 - accuracy: 0.67 - ETA: 0s - loss: 0.6451 - accuracy: 0.72 - 4s 12ms/step - loss: 0.6347 - accuracy: 0.7246 - val_loss: 0.5501 - val_accuracy: 0.8276
Epoch 3/5
345/345 [==============================] - ETA: 2s - loss: 0.5664 - accuracy: 0.70 - ETA: 0s - loss: 0.5918 - accuracy: 0.68 - 4s 11ms/step - loss: 0.5678 - accuracy: 0.7246 - val_loss: 0.4852 - val_accuracy: 0.8276
Epoch 4/5
345/345 [==============================] - ETA: 1s - loss: 0.4838 - accuracy: 0.78 - ETA: 0s - loss: 0.5497 - accuracy: 0.75 - 3s 8ms/step - loss: 0.5700 - accuracy: 0.7246 - val_loss: 0.4857 - val_accuracy: 0.8276
Epoch 5/5
345/345 [============================